In [48]:
import os
from datetime import datetime
import requests as r
import numpy as np
import pandas as pd
import geopandas as gp
from skimage import io
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio as rio
import xarray as xr
import rioxarray as rxr
import rioxarray
import hvplot.xarray
import hvplot.pandas
import json
import panel as pn
import geoviews
import earthaccess
from rasterio.enums import ColorInterp
from rasterio.mask import mask
from rasterio.warp import Resampling
from shapely.geometry import Polygon
from shapely.ops import transform
from tqdm.auto import tqdm
import cupy as cp
import time
import nasa_hls
# There are a couple plots that generate errors that we want to ignore.
import warnings
warnings.filterwarnings('ignore')

In [2]:
earthaccess.login(persist=True)

In [3]:
field = gp.read_file('G:/2024_활동지도/행정경계/korea.shp')
field

,dissolve,geometry
0,1.0,"MULTIPOLYGON (((126.26475 33.11572, 126.26476 ..."


In [4]:
grid = gp.read_file('G:/grid/fin/S2A_OPER_GRID_utm.shp')
grid = grid[['Name', 'geometry']]
tile = grid['Name']
granule_names = list(tile)

In [5]:
bbox = tuple(list(field.total_bounds))
bbox

(124.60971767857728, 33.1123557596338, 131.87278314808853, 38.61370930257595)

In [6]:
year = '2024'

In [7]:
temporal = (f"{year}-05-01T00:00:00", f"{year}-06-15T23:59:59")

In [8]:
earthaccess.search_data

<function earthaccess.api.search_data(count: int = -1, **kwargs: typing_extensions.Any) -> List[earthaccess.results.DataGranule]>

In [9]:
results = []

for name in tqdm(granule_names):
    result = earthaccess.search_data(
        short_name=['HLSL30', 'HLSS30'],
        bounding_box=bbox,
        temporal=temporal,
        count=100,
        cloud_cover=(0, 50),
        granule_name=f'*{name}*'  # 와일드카드로 granule_name에 해당하는 패턴 검색
    )
    print(f"Granule '{name}' - 검색된 개수: {len(result)}")
    results.extend(result)  # 결과를 results 리스트에 추가

  0%|          | 0/36 [00:00<?, ?it/s]

Granule '51SXB' - 검색된 개수: 8
Granule '51SXC' - 검색된 개수: 9
Granule '51SXT' - 검색된 개수: 12
Granule '51SYA' - 검색된 개수: 15
Granule '51SYB' - 검색된 개수: 14
Granule '51SYC' - 검색된 개수: 19
Granule '51SYS' - 검색된 개수: 13
Granule '51SYT' - 검색된 개수: 13
Granule '51SYU' - 검색된 개수: 18
Granule '51SYV' - 검색된 개수: 12
Granule '52SBB' - 검색된 개수: 14
Granule '52SBC' - 검색된 개수: 7
Granule '52SBD' - 검색된 개수: 10
Granule '52SBE' - 검색된 개수: 15
Granule '52SBF' - 검색된 개수: 16
Granule '52SBG' - 검색된 개수: 13
Granule '52SBH' - 검색된 개수: 19
Granule '52SCB' - 검색된 개수: 14
Granule '52SCC' - 검색된 개수: 14
Granule '52SCD' - 검색된 개수: 19
Granule '52SCE' - 검색된 개수: 18
Granule '52SCF' - 검색된 개수: 18
Granule '52SCG' - 검색된 개수: 14
Granule '52SCH' - 검색된 개수: 21
Granule '52SDD' - 검색된 개수: 15
Granule '52SDE' - 검색된 개수: 18
Granule '52SDF' - 검색된 개수: 18
Granule '52SDG' - 검색된 개수: 18
Granule '52SDH' - 검색된 개수: 24
Granule '52SED' - 검색된 개수: 14
Granule '52SEE' - 검색된 개수: 16
Granule '52SEF' - 검색된 개수: 18
Granule '52SEG' - 검색된 개수: 22
Granule '52SFG' - 검색된 개수: 16
Granule '52SGG' -

In [129]:
pd.json_normalize(results).head(5)

,size,meta.concept-type,meta.concept-id,meta.revision-id,meta.native-id,meta.collection-concept-id,meta.provider-id,meta.format,meta.revision-date,umm.TemporalExtent.RangeDateTime.BeginningDateTime,...,umm.CollectionReference.EntryTitle,umm.RelatedUrls,umm.DataGranule.DayNightFlag,umm.DataGranule.Identifiers,umm.DataGranule.ProductionDateTime,umm.DataGranule.ArchiveAndDistributionInformation,umm.Platforms,umm.MetadataSpecification.URL,umm.MetadataSpecification.Name,umm.MetadataSpecification.Version
0,157.224716,granule,G2991239576-LPCLOUD,1,HLS.L30.T51SXB.2024129T021650.v2.0,C2021957657-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-10T07:24:06.306Z,2024-05-08T02:16:50.544Z,...,HLS Landsat Operational Land Imager Surface Re...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.L30.T51SXB.2024129T021650...,2024-05-10T07:19:37.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 16486...","[{'ShortName': 'LANDSAT-9', 'Instruments': [{'...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
1,154.965186,granule,G2995057181-LPCLOUD,1,HLS.S30.T51SXB.2024130T022551.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-11T07:32:05.234Z,2024-05-09T02:37:19.606Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T51SXB.2024130T022551...,2024-05-11T07:27:07.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 16249...","[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
2,160.627707,granule,G3008961922-LPCLOUD,1,HLS.S30.T51SXB.2024135T022529.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-16T08:58:01.379Z,2024-05-14T02:37:16.358Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T51SXB.2024135T022529...,2024-05-16T08:55:08.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 16843...","[{'ShortName': 'Sentinel-2B', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
3,128.251033,granule,G3014314338-LPCLOUD,1,HLS.L30.T51SXB.2024137T021621.v2.0,C2021957657-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-18T07:23:06.795Z,2024-05-16T02:16:21.112Z,...,HLS Landsat Operational Land Imager Surface Re...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.L30.T51SXB.2024137T021621...,2024-05-18T07:19:14.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 13448...","[{'ShortName': 'LANDSAT-8', 'Instruments': [{'...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6
4,176.168429,granule,G3022947928-LPCLOUD,1,HLS.S30.T51SXB.2024140T022531.v2.0,C2021957295-LPCLOUD,LPCLOUD,application/echo10+xml,2024-05-21T13:37:14.768Z,2024-05-19T02:37:18.636Z,...,HLS Sentinel-2 Multi-spectral Instrument Surfa...,[{'URL': 'https://data.lpdaac.earthdatacloud.n...,Day,[{'Identifier': 'HLS.S30.T51SXB.2024140T022531...,2024-05-21T13:34:58.000Z,"[{'Name': 'Not provided', 'SizeInBytes': 18472...","[{'ShortName': 'Sentinel-2A', 'Instruments': [...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,1.6.6


In [12]:
len(results)

549

In [133]:
hls_results_urls = [granule.data_links() for granule in results]

In [13]:
# GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')

In [130]:
f_value = [0,4,16,20,32,36,48,52,64,68,80,84,96,100,112,116,128,132,144,148,160,164,176,180,192,196,208,212,224,228,240,244]

In [131]:
def filter_fmask(fmask, f_values):
    # Filter based on the fmask values in f_values
    mask = np.isin(fmask.data, f_values)
    
    # Apply the mask to the fmask, keeping only the values that are in f_values
    filtered_fmask = xr.where(mask, fmask, np.nan, keep_attrs=True)
    filtered_fmask = filtered_fmask.astype('uint8')
    
    return filtered_fmask

In [132]:
import logging
import os
from rasterio.errors import RasterioIOError  # RasterioIOError 추가

In [ ]:
from tqdm import tqdm  # TQDM 라이브러리 추가

# tqdm을 사용하여 파일 처리 진행 상황 표시
for j, h in enumerate(tqdm(hls_results_urls, desc="Processing files", unit="file")):
    
    outName1 = h[0].split('/')[-1].split('v2.0')[0] +'v2.0_cropped.tif'

    # 밴드 정보 설정
    if h[0].split('/')[4] == 'HLSS30.020':
        bands = ['B02', 'B03', 'B04', 'B8A', 'B11', 'B12','Fmask']  # All band for S30
    else:
        bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07','Fmask']  # All band for L30

    # 밴드 링크 추출
    band_links = [a for a in h if any(b in a for b in bands)]
    
    if os.path.exists(f'G:/2024_활동지도/mask/{year}_indices/{outName1}'):
        print(f"{outName1} has already been processed and is available in this directory, moving to next file.")
        continue    

    try:
        # Dataset 생성
        ds_dict = {}
        
        for band in bands:
            b_links = [l for l in band_links if band in l]
            chunks = dict(band=1, x=1830, y=1830)
            da_list = [rioxarray.open_rasterio(url, chunks=chunks) for url in b_links]
            da = xr.concat(da_list, dim='band', join='override', combine_attrs='drop')
            ds_dict[band] = da
        
        ds = xr.Dataset(ds_dict)

        # Fmask 필터 적용
        fmask_filtered = filter_fmask(ds.Fmask, f_value)
        ds_filtered = xr.where(fmask_filtered, ds, np.nan, keep_attrs=True)

        # 좌표계 변환 및 클리핑
        fsUTM = field.to_crs(ds_filtered.B02.spatial_ref.crs_wkt)
        ds_filtered1 = ds_filtered.rio.clip(fsUTM.geometry.values, fsUTM.crs, all_touched=True)
        ds_filtered1 = ds_filtered1.drop_vars('Fmask')

        # 스케일링 적용
        scaling_factor = 0.0001
        ds_scaled = ds_filtered1 * scaling_factor

        # 파일 저장
        ds_scaled.isel(band=0).rio.to_raster(raster_path=f'G:/2024_활동지도/mask/{year}_indices/{outName1}', driver='COG', compression='LZ77')

        print(f"Processed file {j+1} of {len(hls_results_urls)}")
    
    except Exception as e:
        print(f"Error processing file {outName1}: {e}")
        continue


Processing files:   0%|                                                          | 1/549 [02:56<26:52:05, 176.51s/file]

Processed file 1 of 549


Processing files:   0%|▏                                                         | 2/549 [06:52<32:07:38, 211.44s/file]

Processed file 2 of 549


Processing files:   1%|▎                                                         | 3/549 [10:50<33:53:22, 223.45s/file]

...

Processed file 9 of 549


Processing files:   2%|█                                                        | 10/549 [36:47<31:54:03, 213.07s/file]

Processed file 10 of 549


Processing files:   2%|█▏                                                       | 11/549 [40:28<32:13:41, 215.65s/file]

Processed file 11 of 549
